In [ ]:
'''
Self RAG Implementation
Author: Christian Sarmiento
Purpose: This notebook is intended to get a Self-RAG implementation set up with LangChain/LangGraph.
Date Created: 11/17/24
Last Updated: 11/29/24
Data: Marist College Administrative Corpus Dataset
Sources:
- https://blog.langchain.dev/agentic-rag-with-langgraph/
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_self_rag.ipynb?ref=blog.langchain.dev
- ChatGPT: o1-preview
Note: Most of the code for graph implementation of Self-RAG was taken from the second source.
-----------------------------------------------------------------------------------------------------------------------
RAG Research             |               Machine Learning Independent Study             |              DR. EITEL LAURIA
'''

In [23]:
%pip install langgraph

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.6
    Uninstalling langchain-core-0.3.6:
      Successfully uninstalled langchain-core-0.3.6
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Imports
import sys
sys.path.append("/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Private Code")
from api_keys import openAIKey
from api_keys import langchainKey
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import List, Dict, Optional, Annotated
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda
import operator


import pandas as pd
import os
import gradio as gr  # easy frontend implementation
from pprint import pprint
import numpy as np


from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import SingleTurnSample

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm 

In [2]:
# LangChain Enviornment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langchainKey()
os.environ["OPENAI_API_KEY"] = openAIKey()

In [3]:
# Load Data
csvPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/Cleaned_QA.csv"
maristQA = pd.read_csv(csvPath, header=None)

# To use RecursiveCharacterTextSplitter, we need a list of dictionaries
maristContext = [Document(page_content=text) for text in maristQA[1].tolist()]

In [4]:
# Split Documents into Chunks
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
texts = textSplitter.split_documents(maristContext)

In [5]:
# Store Documents in Vector DB (Chroma)
vectorDB = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

# Setup Retrieval System
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieves 3 documents

In [6]:
# Define Grader class for document grading in Self-RAG
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llmDocGrader = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMGrader = llmDocGrader.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
   tic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
gradePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Grader Chain
retrievalGrader = gradePrompt | structuredLLMGrader

# Testing it
question = "agent memory"
docs = retriever.get_relevant_documents(question)
docTxt = docs[1].page_content
print(retrievalGrader.invoke({"question": question, "document": docTxt}))

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_35902/3869534235.py:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='no'


In [7]:
# Generation Chain

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
ragChain = prompt | llm | StrOutputParser()

# Run
generation = ragChain.invoke({"context": docs, "question": question})
print(generation)

I don't know.


In [8]:
# Define Hallucination Grader Class
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llmHallucinations = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMHallucinationGrader = llmHallucinations.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucinationPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucinationGrader = hallucinationPrompt | structuredLLMHallucinationGrader
hallucinationGrader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='no')

In [9]:
# Answer Grader
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

# LLM with function call
llmAnswerGrading = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMAnswerGrader = llmAnswerGrading.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answerPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

# Chain
answerGrader = answerPrompt | structuredLLMAnswerGrader
answerGrader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

In [10]:
# Question Rewriter
llmRewriter = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
rewritePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

questionRewriter = rewritePrompt | llm | StrOutputParser()
questionRewriter.invoke({"question": question})

'What are the key concepts and techniques related to agent memory in artificial intelligence?'

In [11]:
# Define metric evaluator

## Evaluation LLM & embeddings
evalLLM = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evalEmbeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

## Initialize metrics with LLM and embeddings
contextRecall = LLMContextRecall(llm=evalLLM)
faithfulness = Faithfulness(llm=evalLLM)
factualCorrectness = FactualCorrectness(llm=evalLLM)
semanticSimilarity = SemanticSimilarity(embeddings=evalEmbeddings)

## Collect metrics
evalMetrics = [
    contextRecall,
    faithfulness,
    factualCorrectness,
    semanticSimilarity
]

In [137]:
# Define Graph structure for Self-RAG

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        metrics: evaluation metrics for each generation 
    """
    question: Annotated[str, "question_metadata"]
    generation: str
    documents: List[str]
    metrics: Optional[Dict[str, float]]
    retry_count: int = 0


## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Initialize retry count if not set
    print(f"Current Retry Count: {state["retry_count"]}")

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "retry_count": state["retry_count"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print(f"Current Retry Count: {state['retry_count']}")
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]


    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"Current Retry Count: {state['retry_count']}")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Score each doc
    filtered_docs = []
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"documents": filtered_docs, "question": question, "retry_count": retry_count}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Retry Count: {retry_count}")
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Re-write question
    better_question = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question, "retry_count": retry_count + 1}


## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "generate"
    
    # All documents have been filtered check_relevance
    # We will re-generate a new query
    if not filtered_documents:
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    # We have relevant documents, so generate answer
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print(f"Current Retry Count: {state['retry_count']}")
    
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    score = hallucinationGrader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    
    # Recursion base case
    retry_count = state["retry_count"]
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "useful"
    
    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")

        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answerGrader.invoke({"question": question, "generation": generation})
        grade = score.binary_score

        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    retry_count = state.get("retry_count", 0)
    print(f"Final Retry Count Before Reset: {retry_count}")
    state["retry_count"] = 0
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state

In [138]:
# Build Self-RAG Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery)  
workflow.add_node("evaluateMetrics", evaluateMetrics)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate"
    },
)
workflow.add_edge("transformQuery", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "evaluateMetrics",
        "not useful": "transformQuery",
    },
)
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("evaluateMetrics", END)


# Compile
app = workflow.compile()


In [ ]:
# Inital test run
inputs = {"question": "Who is Carolyn Matheus?"}
async for output in app.astream(inputs):
    for key, value in output.items():

        # Print node
        pprint(f"Node '{key}':")

        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)

        # Print metrics
        if "metrics" in value:
            pprint("Metrics: ")
            pprint(value["metrics"])

    pprint("\n---\n")

# Final generation
pprint("Final Generation: ")
pprint(value["generation"])

# Final metrics
if "metrics" in value:
    pprint("Final Metrics: ")
    pprint(value["metrics"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
'\n---\n'
'Final Generation: '
('Dr. Carolyn C. Matheus is an Associate Professor of Information Systems and '
 'the Director of the Honors Program at Marist College. She holds a PhD in '
 'Organizational Studies with a focus on leadership from SUNY Albany and has '
 'received the National Society of Leadership and Success award for Excellenc

In [139]:
# Put SelfRAG into Gradio
evaluationSamples = []
async def selfRAG(userQuery, history, correctAnswer):
    """
    Gradio-compatible function to process SelfRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        tuple: (chatDisplay, history)
    """

    # Get our input together
    inputs = {"question": userQuery, "retry_count": 0}

    # Start the workflow
    finalOutput = None
    async for output in app.astream(inputs, config={"recursion_limit": 100}):

        # Saving final output for metric purposes
        finalOutput = output

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Display output for gradio
    chatDisplay = [(msg["content"], "User" if msg["role"] == "user" else "LLM") for msg in history]

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })

    

    return history  #, chatDisplay

In [ ]:
# Gradio frontend
interface = gr.Interface(
    fn=selfRAG,
    inputs=[
        gr.Textbox(label="Ask a Question", placeholder="Enter your question here..."),
        gr.State(),  # Keeps track of conversation history
        gr.Textbox(label="Correct Answer (Optional)", placeholder="For evaluation purposes..."),
    ],
    outputs=[
        gr.Chatbot(label="SelfRAG Conversation"),
        gr.State(),  # Updates conversation history
    ],
    title="Self-RAG Implementation",
    description="Interact with the Self-RAG workflow for document-grounded question answering.",
)

# Launch the interface
interface.launch()

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTIO

In [140]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"input_query": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats
    

In [16]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [61]:
# Evaluate our pipeline responses
evalResults = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

{'input_query': 'Who is Carolyn Matheus?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.928909234587568}


In [136]:
# Sample 222 records from our dataset
maristTestSample = maristQA.sample(50, replace=False)
maristTestSample.head()

,0,1
91,Graduate school GPA requirement.,Graduate: FAQsFrequently asked questions about...
398,Interactive media department info,"Film, TV, Games, and Interactive Media Departm..."
123,Who is the writing center director?,Contact InformationAcademic SchoolOfficeEmailP...
182,What is Marist perspective on diversity and in...,"Diversity and Inclusion at MaristAt Marist, we..."
514,Mark James Morreale history?,Contact InformationAcademic SchoolOfficeEmailP...


In [141]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    chatHistory = await selfRAG(row[1][0], chatHistory, row[1][1])
    
    
## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 0
---DECISION: GENERATE---
"Node 'gradeDocuments':"
Current Retry Count: 0
---GENERATE---
---CHECK HALLUCINATIONS---
Current Retry Count: 0
Current Retry Count: 0
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
Final Retry Count Before Reset: 0
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7184164073522444}
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMEN

InvalidUpdateError: At key 'question': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE

# Adding Simple RAG Node
- there are certain questions within our dataset that can cause GraphRecursionErrors, which is an infinite loop within our graph that specifically happens when the process can't come up with a good generation and tries rewriting the query until this error occurs
- In our process above, we have a flag that stops this error and tells the process to generate what it has, which inherently hurts performance since there is a big possibility of us generating on documents that are not relelvant to the query at all. 
- To mitigate this, instead of having the process generate what it has upon reaching the stopping condition, we will add a node the the graph that will run a simple RAG chain if this occurs. 

In [46]:
# Define SimpleRAG chains (taken from Simple RAG notebook in this github repo)

## LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Retriever wrapper to be able to return the retrieved documents from this process
captureRetriever = RunnableLambda(lambda query: retriever.get_relevant_documents(query))
retrievedDocsStorage = {"docs": None}
captureRetriever = RunnableLambda(
    lambda query: (retrievedDocsStorage.update({"docs": retriever.get_relevant_documents(query)})) or retrievedDocsStorage["docs"]
)
    
## Function to format documents into the prompt
def formatDocs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Setup RAG Chain
prompt = hub.pull("rlm/rag-prompt")
simpleRagChain = (
    {"context": captureRetriever | formatDocs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Define Graph structure for Self-RAG

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        metrics: evaluation metrics for each generation 
    """
    question: Annotated[str, operator.add]
    generation: str
    documents: List[str]
    metrics: Optional[Dict[str, float]]
    retry_count: int = 0


## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Initialize retry count if not set
    print(f"Current Retry Count: {state["retry_count"]}")

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "retry_count": state["retry_count"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print(f"Current Retry Count: {state['retry_count']}")
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]


    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"Current Retry Count: {state['retry_count']}")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    if retry_count > 1:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Score each doc
    filtered_docs = []
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"documents": filtered_docs, "question": question, "retry_count": retry_count}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Retry Count: {retry_count}")
    if retry_count > 1:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Re-write question
    betterQuestion = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": betterQuestion, "retry_count": retry_count + 1}

def simpleRAG(state):
    
    print("---SIMPLE RAG---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]


    # RAG generation
    generation = ""
    for chunk in simpleRagChain.stream(question):
        generation += chunk
    
    print(generation)

    # Get retrieved context
    newContext = retrievedDocsStorage["docs"]
    print(f"Context: {newContext}")
    

    # Return updated state
    updatedState = {"documents": newContext, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState
    
## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 1:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "simpleRAG"
    
    # All documents have been filtered check_relevance
    # We will re-generate a new query
    if not filtered_documents:
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    # We have relevant documents, so generate answer
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print(f"Current Retry Count: {state['retry_count']}")
    
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    score = hallucinationGrader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    
    # Recursion base case
    retry_count = state["retry_count"]
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 1:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "useful"
    
    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")

        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answerGrader.invoke({"question": question, "generation": generation})
        grade = score.binary_score

        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    retry_count = state.get("retry_count", 0)
    print(f"Final Retry Count Before Reset: {retry_count}")
    state["retry_count"] = 0
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state

In [48]:
# Build Self-RAG Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery) 
workflow.add_node("simpleRAG", simpleRAG)
workflow.add_node("evaluateMetrics", evaluateMetrics)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate",
        "simpleRAG": "simpleRAG"
    },
)
workflow.add_edge("transformQuery", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "evaluateMetrics",
        "not useful": "transformQuery",
    },
)
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("simpleRAG", "evaluateMetrics")
workflow.add_edge("evaluateMetrics", END)


# Compile
app = workflow.compile()


In [49]:
# Put our process into a function
evaluationSamples = []
ragResults = []
async def selfRAG(userQuery, history, correctAnswer):
    """
    Gradio-compatible function to process SelfRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        list: history
    """

    # Variables
    inputs = {"question": userQuery, "retry_count": 0}
    finalOutput = None
    retryFlag = False

    # Start the workflow
    async for output in app.astream(inputs, config={"recursion_limit": 100}):

        # Saving final output for metric purposes
        finalOutput = output

        # Check if the process went into a loop (generated answer with simpleRAG)
        retryFlag = "simpleRAG" in output

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })
    
    # Save the result of the query with the metrics 
    ragResults.append({
        "question": userQuery,
        "generation": generation,
        "retry": retryFlag
    })

    return history  

In [50]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"question": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats
    

In [51]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [52]:
# Sample records from our dataset
maristTestSample = maristQA.sample(10, replace=False)
maristTestSample.head()

,0,1
141,are classes at marist big?,AcademicsAcademicsNo matter your academic inte...
91,Graduate school GPA requirement.,Graduate: FAQsFrequently asked questions about...
371,Professional Accountancy graduate earnings,Master of Science in Professional Accountancy ...
335,phone applications,"An Entrepreneur for AccessibilityMarch 25, 201..."
617,Graduate Study in the UK,Center for Career Services Department Center f...


In [53]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    print(f"Question: {row[1][0]}")
    chatHistory = await selfRAG(row[1][0], chatHistory, row[1][1])
    
## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

Question: are classes at marist big?
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 0
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'gradeDocuments':"
---TRANSFORM QUERY---
Retry Count: 0
"Node 'transformQuery':"
---RETRIEVE---
Current Retry Count: 1
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 1
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 1
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'gradeDocuments':"
---TRANSFORM QUERY---
Retry Count: 1
"Node 'transformQuery':"
---RETRIEVE---
Current Retry Count: 2
"Node 'retrieve':"
---CHECK DOCUMENT 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 192770 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [182]:
# Make our dataframe of results
processResults = pd.DataFrame(ragResults)
evalResultsDF = pd.DataFrame(evalResults)
finalResultsDF = processResults.merge(evalResultsDF, on="question", how="left")
finalResultsDF.head()

,question,generation,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
0,What are the various Marist School codes?,The retrieved context does not provide specifi...,False,0.000000,0.22,0.6,0.850097
1,Tell me more about the allied health building,"The Allied Health building, completed in Janua...",False,0.894737,0.32,0.0,0.831505
2,email of Dr. Wermuth?,I don't know.,False,0.166667,0.00,0.0,0.740081
3,How long is the IMC degree?,The IMC degree can be completed in three to fo...,False,0.818182,0.35,0.0,0.845312
4,How many states / countries are represented at...,Marist welcomes students from nearly 70 countr...,False,0.250000,0.14,0.0,0.917484


In [ ]:
# Averages & St. devs
print(f"LLM Context Recall - Mean: {finalResultsDF["LLMContextRecall"].mean()}, St. Dev: {finalResultsDF["LLMContextRecall"].std()}")
print(f"FactualCorrectness - Mean: {finalResultsDF["FactualCorrectness"].mean()}, St. Dev: {finalResultsDF["FactualCorrectness"].std()}")
print(f"Faithfulness - Mean: {finalResultsDF["Faithfulness"].mean()}, St. Dev: {finalResultsDF["Faithfulness"].std()}")
print(f"SemanticSimilarity - Mean: {finalResultsDF["SemanticSimilarity"].mean()}, St. Dev: {finalResultsDF["SemanticSimilarity"].std()}")


LLM Context Recall - Mean: 0.42591706539074964, St. Dev: 0.4041121592597289
FactualCorrectness - Mean: 0.20600000000000002, St. Dev: 0.1420563268566381
Faithfulness - Mean: 0.12, St. Dev: 0.2683281572999748
SemanticSimilarity - Mean: 0.8368958429569708, St. Dev: 0.06352500214746996


# Keeping Track of Graph Loops

In [12]:
# Define Graph structure for Self-RAG

def overwriteOperator(_, new_value):
    return new_value

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        metrics: evaluation metrics for each generation 
    """
    question: Annotated[str, operator.add]
    generation: Annotated[str, operator.add]
    documents: Annotated[List[str], overwriteOperator]
    metrics: Annotated[Optional[Dict[str, float]], lambda x, y: {**x, **y} if x and y else x or y]
    retry_count: Annotated[int, operator.add]


## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Initialize retry count if not set
    print(f"Current Retry Count: {state["retry_count"]}")

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "retry_count": state["retry_count"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print(f"Current Retry Count: {state['retry_count']}")
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"Current Retry Count: {state['retry_count']}")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Score each doc
    filtered_docs = []
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"documents": filtered_docs, "question": question, "retry_count": retry_count}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Retry Count: {retry_count}")
    if retry_count > 5:
        print("---MAX RETRY LIMIT REACHED IN TRANSFORM QUERY: FORCING END---")
        return {"documents": state["documents"], "question": state["question"], "retry_count": retry_count}
    
    # Re-write question
    betterQuestion = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": betterQuestion, "retry_count": retry_count + 1}

def graphLoopFallback(state):
    
    print("---GRAPH LOOP - NO GENERATION---")

    '''
    question = state["question"]
    documents = state["documents"]
    retry_count = state["retry_count"]


    # RAG generation
    # Still done because end node (evaluateMetrics) still needs to calculate something
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "retry_count": retry_count}

    return updatedState
    
    '''

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": 0,
        "Faithfulness": 0,
        "FactualCorrectness": 0,
        "SemanticSimilarity": 0,
    }

    return state

async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    retry_count = state.get("retry_count", 0)
    print(f"Final Retry Count Before Reset: {retry_count}")
    state["retry_count"] = 0
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    if "metrics" not in state:
        state["metrics"] = {}
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state
    
## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]
    retry_count = state["retry_count"]

    # Recursion base case
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "graphLoopFallback"
    
    # All documents have been filtered check_relevance
    # We will re-generate a new query
    if not filtered_documents:
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    # We have relevant documents, so generate answer
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print(f"Current Retry Count: {state['retry_count']}")
    
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    score = hallucinationGrader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    
    # Recursion base case
    retry_count = state["retry_count"]
    print(f"Current Retry Count: {retry_count}")
    if retry_count > 5:  
        print("---MAX RETRY LIMIT REACHED: STOPPING---")
        return "graphLoopFallback"
    
    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")

        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answerGrader.invoke({"question": question, "generation": generation})
        grade = score.binary_score

        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"


In [13]:
# Build Self-RAG Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery) 
workflow.add_node("graphLoopFallback", graphLoopFallback)
workflow.add_node("evaluateMetrics", evaluateMetrics)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate",
        "graphLoopFallback": "graphLoopFallback"
    },
)
workflow.add_edge("transformQuery", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "evaluateMetrics",
        "not useful": "transformQuery",
        "graphLoopFallback": "graphLoopFallback"
    },
)
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("graphLoopFallback", END)
workflow.add_edge("evaluateMetrics", END)


# Compile
app = workflow.compile()


In [14]:
# Put our process into a function
evaluationSamples = []
ragResults = []
async def selfRAG(userQuery, history, correctAnswer):
    """
    Gradio-compatible function to process SelfRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        list: history
    """

    # Variables
    inputs = {"question": userQuery, "retry_count": 0}
    finalOutput = None
    retryFlag = False

    # Start the workflow
    async for output in app.astream(inputs, config={"recursion_limit": 100}):

        # Saving final output for metric purposes
        finalOutput = output

        # Check if the process went into a loop (generated answer with simpleRAG)
        if "graphLoopFallback" in output:
            retryFlag = True

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })
    
    # Save the result of the query with the metrics 
    ragResults.append({
        "question": userQuery,
        "generation": generation,
        "ground_truth_response": correctAnswer,
        "retry": retryFlag
    })

    return history  

In [15]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"question": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats
    

In [16]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [17]:
# Sample records from our dataset
maristTestSample = maristQA.sample(100, replace=False)
maristTestSample.head()

,0,1
589,What are the hours of the student financial se...,Transfer Student Admission Department Student ...
656,First Fulbright scholarship awardee?,Scholarships and FellowshipsHelping You Achiev...
543,Witchcraft history class professor?,Contact InformationAcademic SchoolOfficeEmailP...
544,Who's the person to go to for Latin American a...,Contact InformationAcademic SchoolOfficeEmailP...
150,Can I get help with creating my resume and cov...,Center for Career ServicesYour Path to Success...


In [ ]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    print(f"Question: {row[1][0]}")
    chatHistory = await selfRAG(row[1][0], chatHistory, row[1][1])
    
## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

Question: What are the hours of the student financial services office?
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 0
---DECISION: GENERATE---
"Node 'gradeDocuments':"
Current Retry Count: 0
---GENERATE---
---CHECK HALLUCINATIONS---
Current Retry Count: 0
Current Retry Count: 0
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
Final Retry Count Before Reset: 0
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7101235813282704}
Question: First Fulbright scholarship awardee?
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=11da071a-112f-4fb4-b7bc-cdaafe377018,id=11da071a-112f-4fb4-b7bc-cdaafe377018


"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.723360194823954}
Question: Who's the person to go to for Latin American and Caribbean Studies?
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 0
---DECISION: GENERATE---
"Node 'gradeDocuments':"
Current Retry Count: 0
---GENERATE---
---CHECK HALLUCINATIONS---
Current Retry Count: 0
Current Retry Count: 0
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
Final Retry Count Before Reset: 0
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticS

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('502 Server Error: Bad Gateway for url: https://api.smith.langchain.com/runs/batch', '\n<html><head>\n<meta http-equiv="content-type" content="text/html;charset=utf-8">\n<title>502 Server Error</title>\n</head>\n<body text=#000000 bgcolor=#ffffff>\n<h1>Error: Server Error</h1>\n<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>\n<h2></h2>\n</body></html>\n')


"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7430772989476265}
Question: Who made the strategic plan?
---RETRIEVE---
Current Retry Count: 0
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Current Retry Count: 0
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
Current Retry Count: 0
---DECISION: GENERATE---
"Node 'gradeDocuments':"
Current Retry Count: 0
---GENERATE---
---CHECK HALLUCINATIONS---
Current Retry Count: 0
Current Retry Count: 0
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
---EVALUATING METRICS---
Final Retry Count Before Reset: 0
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7269221407735531}
Questi

...

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=78989826-5e8c-4177-b371-e265d3cb383e; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=1b288779-7a3e-4c85-80af-3bd2c95abaa0; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=aef8df22-c194-4d20-8956-53112286f53a; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=4603cf34-8d71-4190-90fc-10126c3e3f5f; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=4ea8f1be-3f39-4059-9bbd-988f7f1be176; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=91fdee8f-7c3c-4d34-a017-949113765db8; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=8172d4bb-cc54-4bf7-8c7a-a8cadf4d1856; trace=20c273ac-3415-4b76-93a9-a36e72f1c422,id=47f05013-d4ab-45e8-a00d-67cf501d021e; trace=20c273ac-3415-4b76

"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7212986010472986}


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=57ae38ed-5197-494b-ab06-174c85150b05; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=dce740ed-ae7c-4f63-8cbe-1b1662803808; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=8b137a34-386e-4508-894c-dc29bce1cb74; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=9ae45b9f-2513-47e9-a61e-9c0ec138579f; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=9ac44f3d-ec98-45b3-88cf-b927f6af8ac4; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=18bb07f4-80c1-4e62-acde-940b31b69a85; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=840a7767-0eae-4abb-b322-29902cb8bb78; trace=6783127f-4df9-4a4b-b611-32423b5c58c4,id=6686711f-f732-4311-96d0-80b212a7c864; trace=6783127f-4df9-4a4b

{'question': 'What are the hours of the student financial services office?', 'LLMContextRecall': 0.5, 'FactualCorrectness': 0.29, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8438083664343583}
{'question': 'First Fulbright scholarship awardee?', 'LLMContextRecall': 0.07692307692307693, 'FactualCorrectness': "Error: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''", 'Faithfulness': nan, 'SemanticSimilarity': 0.69256337670863}
{'question': 'Witchcraft history class professor?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.24, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.9116799600159815}
{'question': "Who's the person to go to for Latin American and Caribbean Studies?", 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.35, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.92530510372576}
{'question': 'Can I get help with creating my resume and cover letters?', 'LLMContextRecall': 0.727

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d,id=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d; trace=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d,id=1d96593b-5342-424b-ac53-238b4259baaa; trace=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d,id=bb48984e-6a18-49d4-ae44-cc93b76b1707; trace=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d,id=e9fc0541-8204-4205-bbcf-309224be7b6e; trace=f186c221-fbe5-41fb-8ea3-2dcf4b8f738d,id=49119470-a162-4a7f-b93b-853605c8b0c2; trace=17afbf3e-9152-4872-935b-9136c7267e5c,id=17afbf3e-9152-4872-935b-9136c7267e5c; trace=ac6907cd-0bb5-45de-90d2-72c71c4c461a,id=ac6907cd-0bb5-45de-90d2-72c71c4c461a; trace=ac6907cd-0bb5-45de-90d2-72c71c4c461a,id=f319074a-ac30-4bc2-8ffb-6ff0a266779a; trace=ac6907cd-0bb5-45de

In [19]:
# Make our dataframe of results
processResults = pd.DataFrame(ragResults)
evalResultsDF = pd.DataFrame(evalResults)
finalResultsDF = processResults.merge(evalResultsDF, on="question", how="left")
finalResultsDF.head()

,question,generation,ground_truth_response,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
0,What are the hours of the student financial se...,The hours of the Student Financial Services of...,Transfer Student Admission Department Student ...,False,0.500000,0.29,0.0,0.843808
1,First Fulbright scholarship awardee?,,Scholarships and FellowshipsHelping You Achiev...,True,0.076923,Error: ufunc 'invert' not supported for the in...,NaN,0.692563
2,Witchcraft history class professor?,The professor for the Witchcraft history class...,Contact InformationAcademic SchoolOfficeEmailP...,False,0.000000,0.24,0.0,0.911680
3,Who's the person to go to for Latin American a...,The person to go to for Latin American and Car...,Contact InformationAcademic SchoolOfficeEmailP...,False,0.000000,0.35,0.0,0.925305
4,Can I get help with creating my resume and cov...,,Center for Career ServicesYour Path to Success...,True,0.727273,Error: ufunc 'invert' not supported for the in...,NaN,0.667127


In [20]:
# Filter out faulty generations (graph loops)
trueRunsDF = finalResultsDF[finalResultsDF["retry"] == False]
trueRunsDF.describe()

,LLMContextRecall,Faithfulness,SemanticSimilarity
count,73.000000,73.000000,73.000000
mean,0.232991,0.017123,0.889489
std,0.318114,0.120248,0.052391
min,0.000000,0.000000,0.706034
25%,0.000000,0.000000,0.858148
50%,0.000000,0.000000,0.897846
75%,0.500000,0.000000,0.933078
max,1.000000,1.000000,0.961005


In [33]:
# Filtering out possible errors with FactualCorrectness computation
excludeString = "Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again."
trueRunsDF = trueRunsDF[trueRunsDF["FactualCorrectness"] != excludeString]
trueRunsDF.describe()

,LLMContextRecall,Faithfulness,SemanticSimilarity
count,72.000000,72.000000,72.000000
mean,0.234243,0.017361,0.890150
std,0.320166,0.121075,0.052451
min,0.000000,0.000000,0.706034
25%,0.000000,0.000000,0.858270
50%,0.000000,0.000000,0.898422
75%,0.500000,0.000000,0.933565
max,1.000000,1.000000,0.961005


In [34]:
# Averages & St. devs
print(f"LLM Context Recall - Mean: {trueRunsDF["LLMContextRecall"].mean()}, St. Dev: {trueRunsDF["LLMContextRecall"].std()}")
print(f"FactualCorrectness - Mean: {trueRunsDF["FactualCorrectness"].mean()}, St. Dev: {trueRunsDF["FactualCorrectness"].std()}")
print(f"Faithfulness - Mean: {trueRunsDF["Faithfulness"].mean()}, St. Dev: {trueRunsDF["Faithfulness"].std()}")
print(f"SemanticSimilarity - Mean: {trueRunsDF["SemanticSimilarity"].mean()}, St. Dev: {trueRunsDF["SemanticSimilarity"].std()}")

LLM Context Recall - Mean: 0.23424272260056572, St. Dev: 0.3201656451589497
FactualCorrectness - Mean: 0.3129166666666667, St. Dev: 0.18389707393761573
Faithfulness - Mean: 0.017361111111111112, St. Dev: 0.12107491634422932
SemanticSimilarity - Mean: 0.8901498617587273, St. Dev: 0.052451479081982055
